<a href="https://colab.research.google.com/github/LC1332/Embed-Adapter/blob/main/notebook/test_100_setence_retrieve_back.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

implemented by [李鲁鲁](https://github.com/LC1332)

- [x] 载入测试数据
- [x] 准备各个embedding函数
- [x] 1nn搜索验证


In [1]:
%cd /content
!rm -rf /content/Embed-Adapter
!git clone https://github.com/LC1332/Embed-Adapter
%cd /content/Embed-Adapter

/content
Cloning into 'Embed-Adapter'...
remote: Enumerating objects: 141, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 141 (delta 9), reused 17 (delta 7), pack-reused 115
Receiving objects: 100% (141/141), 36.80 MiB | 15.16 MiB/s, done.
Resolving deltas: 100% (71/71), done.
/content/Embed-Adapter


In [2]:

!ls data

pseudo_inverses_400.pkl  pseudo_inverses.pkl  test_data.pkl


In [3]:
import pickle
# Load the pickle file
with open('data/test_data.pkl', 'rb') as f:
    test_data = pickle.load(f)

In [4]:
print(test_data.keys())

dict_keys(['texts', 'model2embed'])


In [5]:
print(test_data['model2embed'].keys())

dict_keys(['openai', 'BAAI/bge-large-zh-v1.5', 'BAAI/bge-small-zh-v1.5', 'BAAI/bge-base-zh-v1.5'])


In [6]:
from EmbedAdapter.utils import get_general_embedding


source_model_names = ["BAAI/bge-small-zh-v1.5","BAAI/bge-base-zh-v1.5", "BAAI/bge-large-zh-v1.5"]

source_models = []

for model_name in source_model_names:
    source_models.append({
        "name":model_name,
        "embed_fun":lambda text: get_general_embedding(text, model_name)
    })

In [7]:
target_model_names = ["openai", "BAAI/bge-small-zh-v1.5","BAAI/bge-base-zh-v1.5", "BAAI/bge-large-zh-v1.5"]

为我实现一个python函数 search_index( query, database )

query是一个list of float,

database都是一个list of list of float

将query和database中每一个元素比较cosine距离

返回最近的那个元素对应的index

In [17]:
import numpy as np

def cosine_similarity(vector1, vector2):
    dot_product = np.dot(vector1, vector2)
    norm1 = np.linalg.norm(vector1)
    norm2 = np.linalg.norm(vector2)
    similarity = dot_product / (norm1 * norm2)
    return similarity

def search_index(query, database):
    max_similarity = -1
    closest_index = -1

    for i, vector in enumerate(database):
        similarity = cosine_similarity(query, vector)
        if similarity > max_similarity:
            max_similarity = similarity
            closest_index = i

    return closest_index

In [18]:
# index = search_index(target_embed, target_database)
# print(index)

99


然后考虑去进行测试

test_data中包含text 和 model2embed

text是list of str
model2embed是一个dict，里面的key为'openai', 'BAAI/bge-large-zh-v1.5', 'BAAI/bge-small-zh-v1.5', 'BAAI/bge-base-zh-v1.5'

每一个value都是list of embed_vector，每个embed_vector又是一个list of float

In [21]:
from tqdm import tqdm
from EmbedAdapter import EmbedAdapter

modelpair2score = {}

for source_model in source_models:
    source_model_name = source_model['name']

    print('source = ', source_model_name)

    source_embeds = []

    n = len(test_data['texts'])

    for text in tqdm(test_data['texts']):
        source_embed = get_general_embedding( text, model_name = source_model_name )
        source_embeds.append(source_embed)

    for target_model_name in target_model_names:
        if target_model_name == source_model_name:
            continue
        adapter = EmbedAdapter(source = source_model_name, output = target_model_name)

        target_database = test_data['model2embed'][target_model_name]
        # target_database也是一个长度为n的list, 每个元素是一个list of float 也是embed

        count = 0

        for i in tqdm(range(n)):
            source_embed = source_embeds[i]
            target_embed = adapter(source_embed)

            # 将target_embed 在 target_database中进行cosine度量的1近邻搜索
            index = search_index(target_embed, target_database)

            if index == i:
                count += 1

        score = count / n
        model_pair = (source_model_name, target_model_name)
        modelpair2score[model_pair] = score

source =  BAAI/bge-small-zh-v1.5


100%|██████████| 100/100 [00:08<00:00, 11.63it/s]


source =  BAAI/bge-base-zh-v1.5


  0%|          | 0/100 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/409M [00:00<?, ?B/s]

100%|██████████| 100/100 [00:09<00:00, 10.86it/s]


source =  BAAI/bge-large-zh-v1.5


100%|██████████| 100/100 [00:05<00:00, 16.91it/s]


In [20]:
for source_model in source_models:
    source_model_name = source_model['name']
    for target_model_name in target_model_names:
        if target_model_name == source_model_name:
            continue
        model_pair = (source_model_name, target_model_name)
        score = modelpair2score[model_pair]

0.99


In [16]:
print(len(target_database[0]))

1536


In [23]:
import pandas as pd

# 创建空的数据框作为表格
table = pd.DataFrame(columns=[''] + target_model_names)  # 列标题为目标模型名称

for source_model in source_models:
    source_model_name = source_model['name']
    row = [source_model_name]  # 行标题为源模型名称

    for target_model_name in target_model_names:
        if target_model_name == source_model_name:
            score = pd.NA
        else:
            model_pair = (source_model_name, target_model_name)
            score = modelpair2score[model_pair]
        row.append(score)  # 将得分添加到行中

    # 将行添加到表格中
    table.loc[len(table)] = row

# 打印表格
print(table)

                           openai  BAAI/bge-small-zh-v1.5  \
0  BAAI/bge-small-zh-v1.5    0.99                     NaN   
1   BAAI/bge-base-zh-v1.5    0.99                     1.0   
2  BAAI/bge-large-zh-v1.5    1.00                     1.0   

  BAAI/bge-base-zh-v1.5 BAAI/bge-large-zh-v1.5  
0                   1.0                    1.0  
1                  <NA>                    1.0  
2                   1.0                   <NA>  
